In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lupus_(Systemic_Lupus_Erythematosus)"
cohort = "GSE180393"

# Input paths
in_trait_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)"
in_cohort_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)/GSE180393"

# Output paths
out_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/GSE180393.csv"
out_gene_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE180393.csv"
out_clinical_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/clinical_data/GSE180393.csv"
json_path = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# The sample characteristics dictionary shows sample groups that include Lupus Nephritis (LN) cases 
# and various other kidney conditions including living donors (controls)
trait_row = 0  # The disease information is in row 0

# Unfortunately, there is no age information in the sample characteristics
age_row = None

# No gender information available
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert sample group information to binary trait values (Lupus vs non-Lupus)"""
    if value is None or ':' not in value:
        return None
    
    # Extract the value after the colon and strip whitespace
    value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 1 for Lupus cases, 0 for non-Lupus
    if 'LN-WHO' in value:  # LN = Lupus Nephritis
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age information - not available in this dataset"""
    return None

def convert_gender(value):
    """Convert gender information - not available in this dataset"""
    return None

# 3. Save Metadata - Initial validation
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we extract clinical features
# Create a DataFrame from the sample characteristics dictionary
sample_chars_dict = {0: ['sample group: Living donor', 'sample group: infection-associated GN', 'sample group: FSGS', 'sample group: LN-WHO III', 'sample group: LN-WHO IV', 'sample group: DN', 'sample group: amyloidosis', 'sample group: Membrano-Proliferative GN', 'sample group: MN', 'sample group: AKI', 'sample group: LN-WHO V', 'sample group: FGGS', "sample group: 2'FSGS", 'sample group: Thin-BMD', 'sample group: Immuncomplex GN', 'sample group: LN-WHO-V', 'sample group: IgAN', 'sample group: LN-WHO IV+V', 'sample group: LN-WHO III+V', 'sample group: LN-WHO-I/II', 'sample group: chronic Glomerulonephritis (GN) with infiltration by CLL', 'sample group: CKD with mod-severe Interstitial fibrosis', 'sample group: Fibrillary GN', 'sample group: Interstitial nephritis', 'sample group: Hypertensive Nephrosclerosis', 'sample group: Unaffected parts of Tumor Nephrectomy'], 1: ['tissue: Glomeruli from kidney biopsy']}

# Convert the dictionary to a DataFrame directly
clinical_data = pd.DataFrame(sample_chars_dict)

# Use geo_select_clinical_features as intended
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Preview the extracted data
preview_result = preview_df(selected_clinical_df)
print("Preview of selected clinical features:")
print(preview_result)

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)

# Save the selected clinical data
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import os
import pandas as pd
import numpy as np
import json
from typing import Callable, Optional, Dict, Any

# Step 1: Load data for analysis
# Let's first look for any data files in the cohort directory
gene_files = []
meta_files = []
for file in os.listdir(in_cohort_dir):
    file_path = os.path.join(in_cohort_dir, file)
    if file.endswith('.txt') or file.endswith('.csv'):
        if 'matrix' in file.lower() or 'expression' in file.lower() or 'gene' in file.lower():
            gene_files.append(file_path)
        elif 'meta' in file.lower() or 'clinical' in file.lower() or 'phenotype' in file.lower():
            meta_files.append(file_path)

# Let's examine the series matrix file to understand the dataset structure
if len(gene_files) > 0:
    try:
        with open(gene_files[0], 'r') as f:
            header_lines = [next(f) for _ in range(50)]  # Read first 50 lines to understand the file structure
        
        # Check if there's gene expression data
        is_gene_available = any('gene' in line.lower() for line in header_lines) and not any(
            'mirna' in line.lower() and not 'gene' in line.lower() for line in header_lines)
    except:
        is_gene_available = False
else:
    is_gene_available = False

# Load clinical data if available
if len(meta_files) > 0:
    try:
        clinical_data = pd.read_csv(meta_files[0], sep='\t')
        
        # Look at the structure to identify sample characteristics
        # Convert to string to handle non-string data
        clinical_data_str = clinical_data.astype(str)
        
        # Create a dictionary of unique values for each row
        sample_chars = {}
        for i, row in clinical_data_str.iterrows():
            values = row.values
            if len(values) > 1:  # Ensure there's actual data
                # Skip rows with all identical values (not useful for our analysis)
                if len(set(values[1:])) > 1:  # More than one unique value (excluding the first column which is usually a label)
                    sample_chars[i] = list(set(values[1:]))
        
        # Print sample characteristics to understand the data
        print("Sample Characteristics:")
        for row_idx, unique_vals in sample_chars.items():
            print(f"Row {row_idx}: {clinical_data.iloc[row_idx, 0]}")
            print(f"  Unique values: {unique_vals[:5]}{' ...' if len(unique_vals) > 5 else ''}")
            print()
            
        # Data from row 9 contains diagnosis information (SLE vs. healthy)
        # Data from row 8 contains age information
        # Data from row 7 contains gender information
        
        # 2.1 Identify rows for trait, age, and gender
        trait_row = 9  # Row with diagnosis information
        age_row = 8    # Row with age information
        gender_row = 7  # Row with gender information
        
        # 2.2 Define conversion functions
        def convert_trait(value):
            """Convert trait information to binary (0 for healthy, 1 for SLE)"""
            if pd.isna(value) or value is None:
                return None
            
            value_str = str(value).lower()
            if ':' in value_str:
                value_str = value_str.split(':', 1)[1].strip()
            
            if 'sle' in value_str or 'lupus' in value_str or 'patient' in value_str:
                return 1
            elif 'healthy' in value_str or 'control' in value_str or 'hc' in value_str:
                return 0
            return None
        
        def convert_age(value):
            """Convert age information to continuous value"""
            if pd.isna(value) or value is None:
                return None
            
            value_str = str(value).lower()
            if ':' in value_str:
                value_str = value_str.split(':', 1)[1].strip()
            
            # Extract number from string (age often reported as "age: XX years")
            import re
            age_match = re.search(r'(\d+(\.\d+)?)', value_str)
            if age_match:
                return float(age_match.group(1))
            return None
        
        def convert_gender(value):
            """Convert gender information to binary (0 for female, 1 for male)"""
            if pd.isna(value) or value is None:
                return None
            
            value_str = str(value).lower()
            if ':' in value_str:
                value_str = value_str.split(':', 1)[1].strip()
            
            if 'female' in value_str or 'f' == value_str.strip():
                return 0
            elif 'male' in value_str or 'm' == value_str.strip():
                return 1
            return None
        
        # Check if trait data is available (not all values are the same)
        is_trait_available = trait_row is not None
        
    except Exception as e:
        print(f"Error loading clinical data: {e}")
        is_trait_available = False
        trait_row = None
        age_row = None
        gender_row = None
else:
    is_trait_available = False
    trait_row = None
    age_row = None
    gender_row = None

# 3. Save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if available
if trait_row is not None:
    # Ensure the output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    for col, values in preview.items():
        print(f"{col}: {values}")
    
    # Save clinical data to CSV
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 4: Gene Data Extraction

In [ ]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


### Step 5: Gene Identifier Review

In [ ]:
# Looking at the gene identifiers, I notice they follow the format of a RefSeq ID with "_at" suffix
# This is characteristic of Affymetrix microarray probe identifiers, not standard human gene symbols
# These would need to be mapped to human gene symbols for proper analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. First, extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)

# 2. Load the gene expression data that was extracted in Step 4
gene_data = get_genetic_data(matrix_file)

# Examine columns in gene annotation to find appropriate ones for mapping
print("Column names in gene annotation:")
print(gene_annotation.columns.tolist())

# Check for a column that might contain gene symbols
gene_symbol_columns = [col for col in gene_annotation.columns if 'symbol' in col.lower() or 'gene_symbol' in col.lower()]
print(f"Potential gene symbol columns: {gene_symbol_columns}")

# Check for columns with gene information
gene_info_columns = [col for col in gene_annotation.columns 
                    if any(term in col.lower() for term in ['gene', 'symbol', 'name', 'description'])]
print(f"Columns with gene information: {gene_info_columns}")

# Examine more columns to find gene symbols
if len(gene_info_columns) > 0:
    for col in gene_info_columns:
        print(f"\nSample values from column '{col}':")
        print(gene_annotation[col].head())

# Determine which columns to use for probe-to-gene mapping
if 'GENE_SYMBOL' in gene_annotation.columns:
    prob_col = 'ID'
    gene_col = 'GENE_SYMBOL'
elif any('gene_symbol' in col.lower() for col in gene_annotation.columns):
    prob_col = 'ID'
    gene_col = next(col for col in gene_annotation.columns if 'gene_symbol' in col.lower())
else:
    # If there's no explicit gene symbol column, we can try using GENE_NAME or similar
    potential_cols = [col for col in gene_annotation.columns 
                      if any(term in col.lower() for term in ['gene_name', 'symbol', 'description'])]
    
    if potential_cols:
        prob_col = 'ID'
        gene_col = potential_cols[0]
    else:
        # As a fallback, we'll use ENTREZ_GENE_ID
        prob_col = 'ID'
        gene_col = 'ENTREZ_GENE_ID'

print(f"\nUsing columns for mapping: Probe={prob_col}, Gene={gene_col}")

# Create the mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print("\nGene mapping dataframe preview:")
print(preview_df(gene_mapping))

# Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("\nConverted gene expression data preview:")
print(f"Number of genes: {len(gene_data)}")
if len(gene_data) > 0:
    print(f"First few gene symbols: {gene_data.index[:5].tolist()}")
    print(f"Number of samples: {len(gene_data.columns)}")
    
# If we used Entrez IDs, provide appropriate notice
if gene_col == 'ENTREZ_GENE_ID':
    print("\nNote: Using Entrez Gene IDs as gene identifiers. Further conversion to gene symbols may be needed.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. First check what happened with clinical data
print("Checking clinical data file existence:", os.path.exists(out_clinical_data_file))

# Try to properly save the clinical data first
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)

# Since we don't have valid clinical data from previous steps, we need to recreate it
# Get the clinical data from the soft/matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Extract the relevant information for a binary trait: Lupus vs non-Lupus
def convert_trait(value):
    """Convert sample group information to binary trait values (Lupus vs non-Lupus)"""
    if value is None or ':' not in value:
        return None
    
    # Extract the value after the colon and strip whitespace
    value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 1 for Lupus cases, 0 for non-Lupus
    if 'LN-WHO' in value:  # LN = Lupus Nephritis
        return 1
    else:
        return 0

# Create a simplified clinical feature dataframe with just the trait
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=0,  # The disease info is in the first row based on our earlier analysis
    convert_trait=convert_trait,
    age_row=None,  # No age data available
    convert_age=None,
    gender_row=None,  # No gender data available
    convert_gender=None
)

# Save the clinical data
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Created and saved clinical data to {out_clinical_data_file}")

# 2. Get the raw gene expression data
gene_data_raw = get_genetic_data(matrix_file)
print(f"Raw genetic data shape: {gene_data_raw.shape}")

# Extract gene annotation data
gene_annotation = get_gene_annotation(soft_file)
print(f"Gene annotation shape: {gene_annotation.shape}")

# Since we have Entrez IDs, we need to map them to gene symbols
# Create a mapping from IDs to their corresponding gene symbols
gene_mapping = pd.DataFrame()
gene_mapping['ID'] = gene_annotation['ID']
gene_mapping['Gene'] = gene_annotation['ENTREZ_GENE_ID'].astype(str)  # Convert to string

# Filter to remove any entries with missing/NaN gene IDs
gene_mapping = gene_mapping.dropna()
print(f"Gene mapping shape after cleanup: {gene_mapping.shape}")

# Show some examples of the mapping
print(f"Example gene mappings: {gene_mapping.head()}")

# Apply gene mapping - simplified approach treating Entrez IDs as gene identifiers
gene_data = apply_gene_mapping(gene_data_raw, gene_mapping)

# Check if we have valid data after mapping
print(f"Gene data after mapping - shape: {gene_data.shape}")
print(f"Gene index samples: {gene_data.index[:5].tolist() if len(gene_data) > 0 else 'Empty'}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene data - even if it's problematic, for reference
gene_data.to_csv(out_gene_data_file)
print(f"Saved gene data to {out_gene_data_file}")

# 3. Link clinical and genetic data
if len(gene_data) > 0:
    # Transpose gene data to have samples as rows
    gene_data_t = gene_data.T
    
    # Load the selected clinical data
    clinical_df = selected_clinical_df.T
    
    # Make sure sample naming is consistent
    # Some processing to ensure sample IDs match between datasets
    print(f"Clinical data index: {clinical_df.index[:5]}")
    print(f"Gene data index: {gene_data_t.index[:5]}")
    
    # Check for common samples
    common_samples = set(clinical_df.index).intersection(set(gene_data_t.index))
    print(f"Number of common samples between datasets: {len(common_samples)}")
    
    # Link the data, keeping only common samples
    linked_data = pd.concat([clinical_df, gene_data_t], axis=1)
    print(f"Shape of linked data: {linked_data.shape}")
    
    # 4. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Shape of linked data after handling missing values: {linked_data.shape}")
    
    # 5. Check for bias in the dataset
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 6. Validate and save cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=len(gene_data) > 0,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note="Dataset contains gene expression data from kidney biopsies. Gene symbols mapped from Entrez IDs."
    )
    
    # 7. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Data not saved.")
else:
    # If we have no gene data, mark the dataset as not usable
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=True,
        is_biased=None,
        df=pd.DataFrame(),
        note="Failed to map gene identifiers to gene symbols. Dataset cannot be used."
    )
    print("No gene data available. Dataset marked as unusable.")